# Self-Attentive Sequential Recommender (SASRec) + Supervised Negative Q-Learning (SNQN) Recommender with CQL Loss

In this notebook, we train an SASRec-SNQN model to recommend a list of items to users in the dataset. In this notebook, we are using the H&M dataset to train and evaluate the mdoel. We will compare the success of models with the use of CQL Loss and without it. 

## Setup

In [ ]:
# Clone our repository
!git clone https://github.com/szheng3/recommendation-system.git
# Install Requirements
!pip install trfl

fatal: destination path 'recommendation-system' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## RetailRocket

### Prepare the Data

In [ ]:
# Download the data
!wget https://aipi590.s3.amazonaws.com/events.csv -P 'recommendation-system/Explore_CQL/Data/RR_data'


--2023-05-03 15:49:01--  https://aipi590.s3.amazonaws.com/events.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 52.217.134.25, 52.217.200.9, 52.216.236.163, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|52.217.134.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94237913 (90M) [text/csv]
Saving to: ‘recommendation-system/Explore_CQL/Data/RR_data/events.csv.2’

events.csv.2         85%[================>   ]  76.68M  3.42MB/s    eta 8s     ^C


In [ ]:
# Preprocess the data
!python 'recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer.py' --data='recommendation-system/Explore_CQL/Data/RR_data'

2023-05-03 15:48:48.815614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 15:48:49.905989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Training

#### Without CQL Loss

In [ ]:
!python "recommendation-system/Explore_CQL/DLR2/src/SNQN_v2.py" --model=SASRec --epoch=10 --data="recommendation-system/Explore_CQL/Data/RR_data"

#### With CQL Loss

In [ ]:
!python "recommendation-system/Explore_CQL/DLR2/src/SNQN_v2.py" --model=SASRec --epoch=10 --CQL_alpha=1.0 --data="recommendation-system/Explore_CQL/Data/RR_data"

## H&M

### Prepare the Data

In [ ]:
# Download the data
!wget https://aipi590.s3.amazonaws.com/transactions_train.csv -P "recommendation-system/Explore_CQL/Data/HM_data"

--2023-05-03 15:49:54--  https://aipi590.s3.amazonaws.com/transactions_train.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 54.231.193.185, 52.216.35.73, 3.5.6.116, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|54.231.193.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488002253 (3.2G) [text/csv]
Saving to: ‘recommendation-system/Explore_CQL/Data/HM_data/transactions_train.csv.1’

transactions_train.   0%[                    ]   4.00M  2.26MB/s               ^C


In [14]:
# Preprocess the data
!python "recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer_HM.py" --size=100000 --format=paper --data="recommendation-system/Explore_CQL/Data/HM_data"


Start reading all transaction data ...
Finish reading in 00:00:29

Filter and save all valid sampled data
/content/recommendation-system/Explore_CQL/DLR2/src/gen_replay_buffer_HM.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_sessions['valid_session'] = sampled_sessions.session_id.map(sampled_sessions.groupby('session_id')['item_id'].size() > 2)
Index(['timestamp', 'session_id', 'item_id', 'price', 'sales_channel_id'], dtype='object')

Start counting popularity ...
1049917it [00:39, 26293.69it/s]
Popularity finished in 00:00:40

Start spliting into train, val, test data ...

           Generate Replay Buffer:
                Total Session Size : 100000
                     Train:      70000 ids | 737354 actions
                   

### Training

#### Without CQL Loss

In [15]:
!python "recommendation-system/Explore_CQL/DLR2/src/SNQN_v2_HM.py" --model=SASRec --epoch=10 --data="recommendation-system/Explore_CQL/Data/HM_data"

2023-05-03 17:27:36.365426: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 17:27:37.936418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Not using CQL loss
/content/recommendation-system/Explore_CQL/DLR2/src/SNQN_v2_HM.py:191: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  self.seq = tf.compat.v1.layers.dropout(self.seq,
2023-05-03 17:27:40.471289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype bo

#### With CQL Loss

In [21]:
!python "recommendation-system/Explore_CQL/DLR2/src/SNQN_v2_HM.py" --model=SASRec --epoch=10 --CQL_alpha=0.5 --data="recommendation-system/Explore_CQL/Data/HM_data"

2023-05-03 19:51:48.283485: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 19:51:49.310578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using CQL loss.
/content/recommendation-system/Explore_CQL/DLR2/src/SNQN_v2_HM.py:191: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  self.seq = tf.compat.v1.layers.dropout(self.seq,
2023-05-03 19:51:51.195891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype bool
